In [1]:
using Flux
using Flux: onehot, onecold, logitcrossentropy #, throttle, @epochs
using MLDatasets
using Random
using JLD2
using Statistics
using MLJBase
using BSON: @load # for load weights
using Plots
using Zygote
using Distributions


In [2]:
train_data = JLD2.load_object("data/train_data.jld2");
test_data = JLD2.load_object("data/test_data.jld2");
x_train = JLD2.load_object("data/x_train.jld2");
y_train = JLD2.load_object("data/y_train.jld2");
x_test  = JLD2.load_object("data/x_test.jld2");
y_test  = JLD2.load_object("data/y_test.jld2");

In [3]:
rng =Random.default_rng()

accuracy(model,x,y) = sum(onecold(model(x)) .== onecold(y))/size(x,2)
accuracy(xy, model) = mean( onecold(model(x)) .== onecold(y) for (x,y) in xy)
loss(model) = (x,y)->Flux.logitcrossentropy(model(x), y)
loss(model,x,y) = loss(model)(x, y)
loss(xy, model) = loss(model)(hcat(map(first,xy)...), hcat(map(last,xy)...))

loss (generic function with 3 methods)

In [4]:
FoodNumber = 4
limit = 10
maxCycle = 50

50

In [5]:
model = Chain(Dense(19, 2*19, sigmoid), Dense(2*19, 2))#define the model 
θ, re = Flux.destructure(model);

In [6]:
globalFoodSourceFitness = zeros(Float64, FoodNumber)
globalfoodSourceCount = zeros(Float64, FoodNumber)
globalWeights = [Vector{Float64}(undef,size(θ)[1]) for _ in 1:FoodNumber];

In [7]:
for i in 1:FoodNumber
    for j in 1:size(θ)[1]
        globalWeights[i][j] = rand(Normal(θ[i], 0.05))
    end

end

In [8]:
function calculateFitness(train_data, newWeight)
    return accuracy(train_data, re(newWeight))
end

calculateFitness (generic function with 1 method)

In [43]:
rand()

0.011143756625592305

In [9]:
calculateFitness(train_data, globalWeights[1])

0.47982708933717577

In [37]:
rand(Normal(0, 1))

0.4427625295571819

In [10]:
function updateFoodSource(index, globalFoodSourceFitness, globalWeights)
    localFitness = calculateFitness(train_data, globalWeights[index])
    if globalFoodSourceFitness[index] <= localFitness
        globalFoodSourceFitness[index] = localFitness
        globalfoodSourceCount[index] = 0
    else
        globalfoodSourceCount[index] = globalfoodSourceCount[index]+1
    end
end

updateFoodSource (generic function with 1 method)

In [12]:
for i in 1:FoodNumber
    updateFoodSource(i, globalFoodSourceFitness, globalWeights)
end
    

In [13]:
globalFoodSourceFitness

4-element Vector{Float64}:
 0.47982708933717577
 0.4812680115273775
 0.4812680115273775
 0.4812680115273775

In [14]:
globalfoodSourceCount

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0